In [1]:
%cd /root/DIS/IS-Net/train

/root/DIS/IS-Net/train


In [2]:
import numpy as np
from PIL import Image
import torch
from torch.autograd import Variable
from torchvision import transforms
import torch.nn.functional as F
import os

import requests
import matplotlib.pyplot as plt
from io import BytesIO

# project imports
from data_loader_cache import normalize, im_reader, im_preprocess 
from isnet import *


/root/miniconda3/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class GOSNormalize(object):
    '''
    Normalize the Image using torch.transforms
    '''
    def __init__(self, mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]):
        self.mean = mean
        self.std = std

    def __call__(self,image):
        image = normalize(image,self.mean,self.std)
        return image


transform =  transforms.Compose([GOSNormalize([0.5,0.5,0.5],[1.0,1.0,1.0])])

def load_image(im_path, hypar):
    if im_path.startswith("http"):
        im_path = BytesIO(requests.get(im_path).content)

    im = im_reader(im_path)
    im, im_shp = im_preprocess(im, hypar["cache_size"])
    im = torch.divide(im,255.0)
    shape = torch.from_numpy(np.array(im_shp))
    return transform(im).unsqueeze(0), shape.unsqueeze(0) # make a batch of image, shape


def build_model(hypar,device):
    net = hypar["model"]#GOSNETINC(3,1)

    # convert to half precision
    if(hypar["model_digit"]=="half"):
        net.half()
        for layer in net.modules():
            if isinstance(layer, nn.BatchNorm2d):
                layer.float()

    net.to(device)

    if(hypar["restore_model"]!=""):
        net.load_state_dict(torch.load(hypar["model_path"]+"/"+hypar["restore_model"],map_location=device))
        net.to(device)
    net.eval()
    return net

    
def predict(net,  inputs_val, shapes_val, hypar, device):
    '''
    Given an Image, predict the mask
    '''
    net.eval()

    if(hypar["model_digit"]=="full"):
        inputs_val = inputs_val.type(torch.FloatTensor)
    else:
        inputs_val = inputs_val.type(torch.HalfTensor)

  
    inputs_val_v = Variable(inputs_val, requires_grad=False).to(device) # wrap inputs in Variable
   
    ds_val = net(inputs_val_v)[0] # list of 6 results

    pred_val = ds_val[0][0,:,:,:] # B x 1 x H x W    # we want the first one which is the most accurate prediction

    ## recover the prediction spatial size to the orignal image size
    pred_val = torch.squeeze(F.upsample(torch.unsqueeze(pred_val,0),(shapes_val[0][0],shapes_val[0][1]),mode='bilinear'))

    ma = torch.max(pred_val)
    mi = torch.min(pred_val)
    pred_val = (pred_val-mi)/(ma-mi) # max = 1

    if device == 'cuda': torch.cuda.empty_cache()
    return (pred_val.detach().cpu().numpy()*255).astype(np.uint8) # it is the mask we need

print('ok')

ok


In [4]:
hypar = {} # paramters for inferencing


hypar["model_path"] ="/root/DIS/IS-Net/train/results/training_cloth_segm_dig/checkpoints" ## load trained weights from this path
hypar["restore_model"] = "itr_00024000_dis.pth" ## name of the to-be-loaded weights
hypar["interm_sup"] = False ## indicate if activate intermediate feature supervision

##  choose floating point accuracy --
hypar["model_digit"] = "full" ## indicates "half" or "full" accuracy of float number
hypar["seed"] = 0

hypar["cache_size"] = [768, 768] ## cached input spatial resolution, can be configured into different size

## data augmentation parameters ---
hypar["input_size"] = [768, 768] ## mdoel input spatial size, usually use the same value hypar["cache_size"], which means we don't further resize the images
hypar["crop_size"] = [768, 768] ## random crop size from the input, it is usually set as smaller than hypar["cache_size"], e.g., [920,920] for data augmentation

hypar["model"] = ISNetDIS()

In [5]:
net = build_model(hypar, device)

RuntimeError: Error(s) in loading state_dict for ISNetDIS:
	size mismatch for side1.weight: copying a param with shape torch.Size([4, 64, 3, 3]) from checkpoint, the shape in current model is torch.Size([1, 64, 3, 3]).
	size mismatch for side1.bias: copying a param with shape torch.Size([4]) from checkpoint, the shape in current model is torch.Size([1]).
	size mismatch for side2.weight: copying a param with shape torch.Size([4, 64, 3, 3]) from checkpoint, the shape in current model is torch.Size([1, 64, 3, 3]).
	size mismatch for side2.bias: copying a param with shape torch.Size([4]) from checkpoint, the shape in current model is torch.Size([1]).
	size mismatch for side3.weight: copying a param with shape torch.Size([4, 128, 3, 3]) from checkpoint, the shape in current model is torch.Size([1, 128, 3, 3]).
	size mismatch for side3.bias: copying a param with shape torch.Size([4]) from checkpoint, the shape in current model is torch.Size([1]).
	size mismatch for side4.weight: copying a pa

In [ ]:
image_path = "https://aiyongaizjk.oss-cn-zhangjiakou.aliyuncs.com/datasets/child/paint/1581.png"
image_bytes = BytesIO(requests.get(image_path).content)

image_tensor, orig_size = load_image(image_path, hypar) 
m0,m1,m2,m3,m4,m5,m6 = predict(net,image_tensor,orig_size, hypar, device)

mask = m0

f, ax = plt.subplots(1,2, figsize = (35,20))
img_array = np.array(Image.open(image_bytes))

# nnp = np.concatenate((img_array, mask[:, :, np.newaxis]), axis=2)
nnp = img_array * (mask > 110)[:, :, np.newaxis]

ax[0].imshow(img_array) # Original image
ax[1].imshow(nnp) # retouched image

ax[0].set_title("Original Image")
ax[1].set_title("Mask")

plt.show()